# LangGraph 记忆对话最小实现

这是一个使用 LangGraph 实现记忆对话的最简单示例。

## 特点
- 自动管理对话历史
- 支持多用户会话
- 状态持久化
- 代码简洁易懂

## 1. 安装依赖

In [ ]:
# 安装必要的包
# !pip install langgraph langchain-ollama langchain-core

## 2. 导入库

In [ ]:
from typing import Annotated, List
from typing_extensions import TypedDict

# LangGraph imports
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver

# LangChain imports
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_ollama import OllamaLLM

print("✅ 库导入成功")

## 3. 配置模型

In [ ]:
# 配置 Ollama
OLLAMA_BASE_URL = "http://localhost:11434"
OLLAMA_MODEL = "gemma3:4b"  # 或者你安装的其他模型

# 创建 LLM 实例
llm = OllamaLLM(
    base_url=OLLAMA_BASE_URL,
    model=OLLAMA_MODEL,
    temperature=0.7
)

# 测试连接
try:
    test_response = llm.invoke("Hello")
    print(f"✅ Ollama 连接成功，模型: {OLLAMA_MODEL}")
    print(f"测试响应: {test_response[:50]}...")
except Exception as e:
    print(f"❌ Ollama 连接失败: {e}")
    print("请确保 Ollama 正在运行: ollama serve")
    print(f"并安装模型: ollama pull {OLLAMA_MODEL}")

## 4. 定义状态

In [ ]:
# 定义 LangGraph 状态
class State(TypedDict):
    """对话状态定义"""
    messages: Annotated[List[BaseMessage], add_messages]

print("✅ 状态定义完成")

## 5. 创建聊天节点

In [ ]:
def chatbot_node(state: State):
    """聊天机器人节点 - 处理用户消息并生成回复"""
    
    # 创建系统消息
    system_message = SystemMessage(content="""你是一个友好的AI助手。
你能记住对话历史，并基于之前的对话内容进行回复。
请保持友好、有帮助的态度。""")
    
    # 准备消息列表：系统消息 + 历史消息
    messages = [system_message] + state["messages"]
    
    # 调用 LLM 生成回复
    response = llm.invoke(messages)
    
    # 返回 AI 消息
    return {"messages": [AIMessage(content=response)]}

print("✅ 聊天节点创建完成")

## 6. 构建图和应用

In [10]:
def create_chat_app():
    """创建 LangGraph 聊天应用"""
    
    # 创建状态图
    graph = StateGraph(State)
    
    # 添加节点
    graph.add_node("chatbot", chatbot_node)
    
    # 添加边：START -> chatbot -> END
    graph.add_edge(START, "chatbot")
    graph.add_edge("chatbot", END)
    
    # 添加记忆检查点
    memory = MemorySaver()
    
    # 编译图
    app = graph.compile(checkpointer=memory)
    print(app.get_graph().draw_mermaid())
    return app

# 创建聊天应用
chat_app = create_chat_app()

print("✅ LangGraph 聊天应用创建成功")
print("\n🎯 LangGraph 记忆特点:")
print("- 自动管理对话历史")
print("- 支持多用户、多会话")
print("- 可以保存和恢复对话状态")
print("- 支持复杂的状态管理")

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	chatbot(chatbot)
	__end__([<p>__end__</p>]):::last
	__start__ --> chatbot;
	chatbot --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc

✅ LangGraph 聊天应用创建成功

🎯 LangGraph 记忆特点:
- 自动管理对话历史
- 支持多用户、多会话
- 可以保存和恢复对话状态
- 支持复杂的状态管理


## 7. 基础聊天演示

In [6]:
def chat(message: str, session_id: str = "default"):
    """发送消息并获取回复"""
    
    # 配置会话
    config = {"configurable": {"thread_id": session_id}}
    
    # 调用聊天应用
    result = chat_app.invoke(
        {"messages": [HumanMessage(content=message)]},
        config=config
    )
    
    # 返回 AI 回复
    return result["messages"][-1].content

# 测试对话
print("🤖 开始对话演示...\n")

# 第一轮对话
user_msg1 = "你好，我叫Alice，是一名数据科学家"
ai_response1 = chat(user_msg1, "demo_session")
print(f"👤 用户: {user_msg1}")
print(f"🤖 AI: {ai_response1}\n")

# 第二轮对话
user_msg2 = "我在研究机器学习算法"
ai_response2 = chat(user_msg2, "demo_session")
print(f"👤 用户: {user_msg2}")
print(f"🤖 AI: {ai_response2}\n")

# 第三轮对话 - 测试记忆
user_msg3 = "你还记得我的名字吗？"
ai_response3 = chat(user_msg3, "demo_session")
print(f"👤 用户: {user_msg3}")
print(f"🤖 AI: {ai_response3}\n")

# 第四轮对话 - 测试记忆
user_msg4 = "我的职业是什么？"
ai_response4 = chat(user_msg4, "demo_session")
print(f"👤 用户: {user_msg4}")
print(f"🤖 AI: {ai_response4}")

🤖 开始对话演示...

👤 用户: 你好，我叫Alice，是一名数据科学家
🤖 AI: 你好Alice！很高兴认识你！作为一名数据科学家，听起来你一定很棒！有什么我可以帮助你的吗？比如，你现在在做什么项目，或者有什么想聊的？ 😊


👤 用户: 我在研究机器学习算法
🤖 AI: 你好Alice！很高兴听到你正在研究机器学习算法！这真是个令人兴奋的领域。 你目前主要关注哪些机器学习算法呢？ 是深度学习、决策树、支持向量机，还是其他类型的算法？ 

或者，你对机器学习的哪个方面特别感兴趣？ 例如，模型选择、特征工程、模型评估，还是算法的理论基础？  😊  告诉我你现在在研究什么，我可以尽力提供帮助，或者一起探讨一些有趣的知识！


👤 用户: 你还记得我的名字吗？
🤖 AI: 你好Alice！当然记得！很高兴再次见到你！ 😊 你在研究机器学习算法，真是个不错的选择。 

你现在主要在研究哪些算法呢？ 还是说有什么想聊的？


👤 用户: 我的职业是什么？
🤖 AI: 你好Alice！当然记得！很高兴再次见到你！😊 你在研究机器学习算法，真是个不错的选择。

你现在主要在研究哪些算法呢？ 还是说有什么想聊的？

你之前告诉我你是一名数据科学家，所以你的职业是数据科学家！ 很高兴再次和你交流！ 😊 



## 8. 查看对话历史

In [7]:
def get_conversation_history(session_id: str = "default"):
    """获取对话历史"""
    config = {"configurable": {"thread_id": session_id}}
    
    # 获取当前状态
    state = chat_app.get_state(config)
    
    return state.values.get("messages", [])

# 显示完整对话历史
print("📜 完整对话历史:")
print("=" * 40)

history = get_conversation_history("demo_session")

for i, msg in enumerate(history, 1):
    if isinstance(msg, HumanMessage):
        print(f"{i}. 👤 用户: {msg.content}")
    elif isinstance(msg, AIMessage):
        print(f"{i}. 🤖 AI: {msg.content}")
    print()

print(f"📊 总消息数: {len(history)}")

📜 完整对话历史:
1. 👤 用户: 你好，我叫Alice，是一名数据科学家

2. 🤖 AI: 你好Alice！很高兴认识你！作为一名数据科学家，听起来你一定很棒！有什么我可以帮助你的吗？比如，你现在在做什么项目，或者有什么想聊的？ 😊


3. 👤 用户: 我在研究机器学习算法

4. 🤖 AI: 你好Alice！很高兴听到你正在研究机器学习算法！这真是个令人兴奋的领域。 你目前主要关注哪些机器学习算法呢？ 是深度学习、决策树、支持向量机，还是其他类型的算法？ 

或者，你对机器学习的哪个方面特别感兴趣？ 例如，模型选择、特征工程、模型评估，还是算法的理论基础？  😊  告诉我你现在在研究什么，我可以尽力提供帮助，或者一起探讨一些有趣的知识！


5. 👤 用户: 你还记得我的名字吗？

6. 🤖 AI: 你好Alice！当然记得！很高兴再次见到你！ 😊 你在研究机器学习算法，真是个不错的选择。 

你现在主要在研究哪些算法呢？ 还是说有什么想聊的？


7. 👤 用户: 我的职业是什么？

8. 🤖 AI: 你好Alice！当然记得！很高兴再次见到你！😊 你在研究机器学习算法，真是个不错的选择。

你现在主要在研究哪些算法呢？ 还是说有什么想聊的？

你之前告诉我你是一名数据科学家，所以你的职业是数据科学家！ 很高兴再次和你交流！ 😊 


📊 总消息数: 8


## 9. 多用户会话演示

In [8]:
print("👥 多用户会话演示\n")

# 用户A的对话
print("=== 用户A的会话 ===")
response_a1 = chat("你好，我是Alice，我喜欢画画", "user_a")
print(f"👤 Alice: 你好，我是Alice，我喜欢画画")
print(f"🤖 AI: {response_a1}\n")

# 用户B的对话
print("=== 用户B的会话 ===")
response_b1 = chat("嗨，我是Bob，我是程序员", "user_b")
print(f"👤 Bob: 嗨，我是Bob，我是程序员")
print(f"🤖 AI: {response_b1}\n")

# 继续用户A的对话
print("=== 继续Alice的会话 ===")
response_a2 = chat("你还记得我的爱好吗？", "user_a")
print(f"👤 Alice: 你还记得我的爱好吗？")
print(f"🤖 AI: {response_a2}\n")

# 继续用户B的对话
print("=== 继续Bob的会话 ===")
response_b2 = chat("我的职业是什么？", "user_b")
print(f"👤 Bob: 我的职业是什么？")
print(f"🤖 AI: {response_b2}")

👥 多用户会话演示

=== 用户A的会话 ===
👤 Alice: 你好，我是Alice，我喜欢画画
🤖 AI: 你好 Alice！很高兴认识你！画画真棒！你喜欢画什么呢？是风景、人物、抽象画还是其他什么？ 


=== 用户B的会话 ===
👤 Bob: 嗨，我是Bob，我是程序员
🤖 AI: 嗨 Bob！ 很高兴认识你。作为程序员，你一定很喜欢解决问题和创造东西吧？有什么我可以帮你的吗？比如，你想聊聊你最近在做的项目，或者有什么编程问题想讨论？ 😊


=== 继续Alice的会话 ===
👤 Alice: 你还记得我的爱好吗？
🤖 AI: 当然记得！你喜欢画画，对吗？你喜欢画什么呢？是风景、人物、抽象画还是其他什么？ 


=== 继续Bob的会话 ===
👤 Bob: 我的职业是什么？
🤖 AI: 嗨 Bob！很高兴认识你。作为程序员，你一定很喜欢解决问题和创造东西吧？ 

根据之前的对话，你是一名程序员。 😊 

你最近在做什么项目呢？或者有什么编程问题想讨论吗？



## 10. 交互式聊天

In [ ]:
def interactive_chat():
    """交互式聊天函数"""
    print("🎯 交互式聊天开始！")
    print("输入 'quit' 退出，输入 'history' 查看历史")
    print("=" * 40)
    
    session_id = "interactive_session"
    
    while True:
        try:
            # 获取用户输入
            user_input = input("\n👤 你: ").strip()
            
            if user_input.lower() == 'quit':
                print("👋 再见！")
                break
            elif user_input.lower() == 'history':
                history = get_conversation_history(session_id)
                print(f"\n📜 对话历史 (共{len(history)}条消息):")
                for msg in history[-6:]:  # 显示最近6条
                    role = "👤" if isinstance(msg, HumanMessage) else "🤖"
                    print(f"{role} {msg.content}")
                continue
            elif not user_input:
                continue
            
            # 获取AI回复
            response = chat(user_input, session_id)
            print(f"🤖 AI: {response}")
            
        except KeyboardInterrupt:
            print("\n\n👋 再见！")
            break
        except Exception as e:
            print(f"❌ 出错了: {e}")

# 运行交互式聊天
# 注意：在Jupyter中运行可能有输入限制，建议在终端中运行
print("💡 提示：如果要进行交互式聊天，请取消下面的注释")
# interactive_chat()

## 11. 输出图的Mermaid

In [ ]:
def print_graph_mermaid():
    """输出LangGraph的Mermaid图表示"""
    try:
        # 获取图的Mermaid表示
        mermaid_code = chat_app.get_graph().draw_mermaid()
        print("🎨 LangGraph Mermaid 图:")
        print("=" * 40)
        print(mermaid_code)
        print("=" * 40)
        return mermaid_code
    except Exception as e:
        print(f"❌ 生成Mermaid图失败: {e}")
        return None

# 输出图的Mermaid表示
mermaid_graph = print_graph_mermaid()

## 12. 总结

### 🎉 LangGraph 记忆对话的优势

1. **简单易用**: 只需几行代码就能实现记忆对话
2. **自动管理**: 无需手动管理对话历史
3. **多用户支持**: 天然支持多用户、多会话隔离
4. **状态持久化**: 自动保存和恢复对话状态
5. **扩展性强**: 可以轻松添加更多功能

### 🔧 核心组件

- **State**: 定义对话状态结构
- **Node**: 处理逻辑的节点函数
- **Graph**: 工作流图结构
- **Checkpointer**: 状态持久化机制

### 🚀 下一步

- 添加消息修剪和摘要功能
- 集成更多工具和功能
- 部署为API服务
- 添加用户认证和权限管理

In [ ]:
print("🎊 LangGraph 记忆对话最小实现演示完成！")
print("\n📚 更多示例请查看:")
print("- examples/langgraph_memory_example.py")
print("- examples/async_langgraph_memory.py")
print("- examples/fastapi_langgraph_memory.py")

## 13. 单元格完整实现 - 最简记忆对话

In [5]:
# LangGraph 最简记忆对话 - 完整实现（单个单元格）
from typing import Annotated, List
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_ollama import OllamaLLM

# 1. 配置模型
llm = OllamaLLM(base_url="http://localhost:11434", model="gemma3:4b", temperature=0.7)

# 2. 定义状态
class State(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]

# 3. 创建聊天节点
def chatbot_node(state: State):
    system_msg = SystemMessage(content="你是一个友好的AI助手，能记住对话历史。")
    messages = [system_msg] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": [AIMessage(content=response)]}

# 4. 构建图和应用
graph = StateGraph(State)
graph.add_node("chatbot", chatbot_node)
graph.add_edge(START, "chatbot")
graph.add_edge("chatbot", END)
app = graph.compile(checkpointer=MemorySaver())

# 5. 聊天函数
def chat(message: str, session_id: str = "default"):
    config = {"configurable": {"thread_id": session_id}}
    result = app.invoke({"messages": [HumanMessage(content=message)]}, config=config)
    return result["messages"][-1].content




# 6. 测试对话
print("🤖 LangGraph 最简记忆对话测试:")
print("=" * 40)

# 测试记忆功能
response1 = chat("你好，我叫Alice，是程序员", "test")
print(f"👤 用户: 你好，我叫Alice，是程序员")
print(f"🤖 AI: {response1}\n")

response2 = chat("你还记得我的名字和职业吗？", "test")
print(f"👤 用户: 你还记得我的名字和职业吗？")
print(f"🤖 AI: {response2}")

print("\n✅ 完成！仅用30行代码实现记忆对话")

🤖 LangGraph 最简记忆对话测试:
👤 用户: 你好，我叫Alice，是程序员
🤖 AI: 你好 Alice，很高兴认识你！我是你的AI助手。作为程序员，有什么我可以帮助你的吗？比如，你想聊聊编程，或者需要一些技术方面的建议？


👤 用户: 你还记得我的名字和职业吗？
🤖 AI: 是的，我记得你叫Alice，是一名程序员。很高兴再次见到你！ 😊


✅ 完成！仅用30行代码实现记忆对话


## 15. Redis存储实现

### 方法1: 使用官方Redis Checkpointer (推荐)

## 18. Redis存储最简实现（单元格版本）

In [16]:
# LangGraph + Redis 最简记忆对话（单个单元格完整实现）
from typing import Annotated, List
from typing_extensions import TypedDict
import redis
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_ollama import OllamaLLM
from langgraph.checkpoint.redis import RedisSaver


# 1. Redis配置和连接
try:
    redis_client = redis.from_url("redis://localhost:6379/0")
    redis_client.ping()
    print("✅ Redis连接成功")

    # 2. 状态定义
    class State(TypedDict):
        messages: Annotated[List[BaseMessage], add_messages]

    # 3. 聊天节点
    def chat_node(state: State):
        llm = OllamaLLM(base_url="http://localhost:11434", model="gemma3:4b", temperature=0.7)
        system_msg = SystemMessage(content="你是AI助手，数据存储在Redis中。")
        messages = [system_msg] + state["messages"]
        response = llm.invoke(messages)
        return {"messages": [AIMessage(content=response)]}

    # 4. 构建Redis应用
    graph = StateGraph(State)
    graph.add_node("chat", chat_node)
    graph.add_edge(START, "chat")
    graph.add_edge("chat", END)

    # 修复RedisSaver初始化错误
    try:
        # 方法1: 使用连接字符串
        redis_app = graph.compile(checkpointer=RedisSaver.from_conn_string("redis://localhost:6379/0"))
        print("✅ 使用连接字符串初始化RedisSaver")
    except AttributeError:
        try:
            # 方法2: 传递redis参数
            redis_app = graph.compile(checkpointer=RedisSaver(redis=redis_client))
            print("✅ 使用redis参数初始化RedisSaver")
        except Exception:
            try:
                # 方法3: 使用URL字符串
                redis_app = graph.compile(checkpointer=RedisSaver("redis://localhost:6379/0"))
                print("✅ 使用URL字符串初始化RedisSaver")
            except Exception:
                # 回退到内存存储
                from langgraph.checkpoint.memory import MemorySaver
                redis_app = graph.compile(checkpointer=MemorySaver())
                print("⚠️ RedisSaver初始化失败，回退到内存存储")

    # 5. 聊天函数
    def redis_chat(message: str, session_id: str = "default"):
        config = {"configurable": {"thread_id": session_id}}
        result = redis_app.invoke({"messages": [HumanMessage(content=message)]}, config=config)
        return result["messages"][-1].content

    # 6. 测试Redis记忆对话
    print("\n🤖 Redis记忆对话测试:")
    print("=" * 30)

    # 测试对话
    r1 = redis_chat("你好，我是赵六，是教师", "redis_demo")
    print(f"👤 用户: 你好，我是赵六，是教师")
    print(f"🤖 AI: {r1}\n")

    r2 = redis_chat("我的职业是什么？", "redis_demo")
    print(f"👤 用户: 我的职业是什么？")
    print(f"🤖 AI: {r2}\n")

    # 检查Redis存储
    keys = redis_client.keys("*redis_demo*")
    print(f"📊 Redis存储键数: {len(keys)}")
    print("✅ Redis记忆对话成功！仅25行代码实现持久化记忆")

except redis.ConnectionError:
    print("❌ Redis连接失败，请启动Redis服务")
    print("安装: pip install redis")
    print("启动: redis-server")
except ImportError:
    print("❌ 请安装: pip install langgraph-checkpoint-redis redis")
except Exception as e:
    print(f"❌ 错误: {e}")


🤖 Memory Chat Test:
✅ Redis connection successful


RedisSearchError: Unexpected error while searching: Required Redis db module search >= 20600 OR searchlight >= 20600 not installed. See Redis Stack docs at https://redis.io/docs/latest/operate/oss_and_stack/install/install-stack/.

In [20]:
#
# LangGraph + Redis 最小实现记忆对话
from typing import Annotated, List
from typing_extensions import TypedDict
import redis
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.redis import RedisSaver

# 1. 定义状态
class State(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]

# 2. 创建聊天节点
def chat_node(state: State):
    # 使用 OpenAI 替代 Ollama
    llm = OllamaLLM(model="gemma3:4b", temperature=0.7)
    system_msg = SystemMessage(content="你是AI助手，记忆存储在Redis中。")
    messages = [system_msg] + state["messages"]
    response = llm.invoke(messages)
    return {"messages": [AIMessage(content=response.content)]}

# 3. 构建图
graph = StateGraph(State)
graph.add_node("chat", chat_node)
graph.add_edge(START, "chat")
graph.add_edge("chat", END)

# 4. 连接Redis并创建应用
try:
    # 创建Redis客户端
    redis_client = redis.from_url("redis://localhost:6379/0")
    redis_client.ping()  # 测试连接
    
    # 使用RedisSaver初始化
    redis_app = graph.compile(checkpointer=RedisSaver(redis_client=redis_client))
    print("✅ Redis连接成功")
    
    # 5. 聊天函数
    def chat(message: str, session_id: str = "default"):
        config = {"configurable": {"thread_id": session_id}}
        result = redis_app.invoke({"messages": [HumanMessage(content=message)]}, config=config)
        return result["messages"][-1].content
    
    # 6. 测试对话
    print("\n🤖 Redis记忆对话测试:")
    print("=" * 30)
    
    # 第一轮对话
    r1 = chat("你好，我是张三，我是一名软件工程师", "user_session_1")
    print(f"👤 用户: 你好，我是张三，我是一名软件工程师")
    print(f"🤖 AI: {r1}\n")
    
    # 第二轮对话 - 测试记忆
    r2 = chat("你还记得我的职业吗？", "user_session_1")
    print(f"👤 用户: 你还记得我的职业吗？")
    print(f"🤖 AI: {r2}\n")
    
    # 检查Redis存储
    keys = redis_client.keys("*user_session_1*")
    print(f"📊 Redis存储键数: {len(keys)}")
    print("✅ Redis记忆对话成功实现！")
    
except redis.ConnectionError:
    print("❌ Redis连接失败，请确保Redis服务已启动")
    print("安装: pip install redis")
    print("启动: redis-server")
except ImportError:
    print("❌ 请安装必要的包: pip install langgraph-checkpoint-redis redis")
except Exception as e:
    print(f"❌ 错误: {str(e)}")
    
    # 回退到内存存储
    from langgraph.checkpoint.memory import MemorySaver
    memory_app = graph.compile(checkpointer=MemorySaver())
    print("⚠️ 回退到内存存储")
    
    def chat_memory(message: str, session_id: str = "default"):
        config = {"configurable": {"thread_id": session_id}}
        result = memory_app.invoke({"messages": [HumanMessage(content=message)]}, config=config)
        return result["messages"][-1].content
    
    # 测试内存存储
    print("\n🤖 内存记忆对话测试:")
    print("=" * 30)
    
    m1 = chat_memory("你好，我是李四，我是一名教师", "memory_session")
    print(f"👤 用户: 你好，我是李四，我是一名教师")
    print(f"🤖 AI: {m1}\n")
    
    m2 = chat_memory("你还记得我的职业吗？", "memory_session")
    print(f"👤 用户: 你还记得我的职业吗？")
    print(f"🤖 AI: {m2}\n")
    
    print("✅ 内存记忆对话成功实现！")

✅ Redis连接成功

🤖 Redis记忆对话测试:
❌ 错误: Unexpected error while searching: Required Redis db module search >= 20600 OR searchlight >= 20600 not installed. See Redis Stack docs at https://redis.io/docs/latest/operate/oss_and_stack/install/install-stack/.
⚠️ 回退到内存存储

🤖 内存记忆对话测试:
00:31:54 httpx INFO   HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


AttributeError: 'str' object has no attribute 'content'